In [333]:
import pandas as pd

In [334]:
data_allocine = pd.read_csv("input_datasets/allocine.csv")
data_imdb = pd.read_csv("input_datasets/imdbmovies.csv")




In [335]:
# display(data_allocine.info())

In [336]:
# display(data_imdb.info())

formatage durée :

In [337]:
import pandas as pd

# Charger le dataset
data_imdb = pd.read_csv("input_datasets/imdbmovies.csv")

# Fonction pour vérifier si la durée est dans le format spécifié
def is_valid_timing(timing):
    return 'h' in timing and 'm' in timing

# Afficher les durées qui ne sont pas dans le format spécifié
invalid_timings = data_imdb[~data_imdb['duree'].apply(is_valid_timing)]
print("Durées qui ne sont pas dans le format spécifié 'xh ym':")
print(invalid_timings[['duree']])


Durées qui ne sont pas dans le format spécifié 'xh ym':
     duree
0        0
5        0
17       0
18       0
24       0
...    ...
2465    2h
2466     0
2470     0
2478    2h
2493    2h

[221 rows x 1 columns]


In [338]:
def convert_timing(timing):
    try:
        if 'h' in timing and 'm' in timing:
            timing_cleaned = timing.replace('m', '')  # Supprimer 'm' de la chaîne
            hours, minutes = map(int, timing_cleaned.split('h '))
            total_minutes = hours * 60 + minutes
        elif 'h' in timing:
            hours = int(timing.replace('h', ''))
            total_minutes = hours * 60
        elif 'm' in timing:
            total_minutes = int(timing.replace('m', ''))
        else:
            total_minutes = 0
        return total_minutes
    except:
        return None


data_imdb['duree']= data_imdb['duree'].apply(convert_timing)


formatage date :


In [339]:
mois_numeriques = {
    "janvier": "01",
    "février": "02",
    "mars": "03",
    "avril": "04",
    "mai": "05",
    "juin": "06",
    "juillet": "07",
    "août": "08",
    "septembre": "09",
    "octobre": "10",
    "novembre": "11",
    "décembre": "12"
}

# Fonction pour convertir le format de date pour allocine
def format_date_allocine(date_string):

    for m in mois_numeriques.keys():
        if m in date_string:
            splits = date_string.split()
            day = str(splits[0])
            month = str(mois_numeriques[splits[1]])
            year = str(splits[-1])
            date_format = day + "/" + month + "/" + year
            return date_format

    date_format = str(date_string.split(' au ')[0])
    return date_format

# Fonction pour convertir le format de date pour imdb
def convert_date_format(date_str):
    try:
        return pd.to_datetime(date_str).strftime('%d/%m/%Y')
    except:
        return None

# Convertir les dates du csv d'allocine
data_allocine['semaine_fr'] = data_allocine['semaine_fr'].apply(format_date_allocine)

# Appliquer la fonction pour convertir le format de date
data_imdb['date_sortie_y'] = data_imdb['date_sortie_y'].apply(convert_date_format)

# Supprime les données avec no date found / None
data_imdb = data_imdb[(data_imdb["date_sortie_y"] != 'No date found') & (data_imdb["date_sortie_y"].notna())]

# Afficher les premières lignes pour vérification
display(data_imdb["date_sortie_y"].info())

# Supprimer les données avec dates antérieurs à 2000

<class 'pandas.core.series.Series'>
Index: 2245 entries, 1 to 2499
Series name: date_sortie_y
Non-Null Count  Dtype 
--------------  ----- 
2245 non-null   object
dtypes: object(1)
memory usage: 35.1+ KB


None

Gestion du Budget:

In [340]:
data_imdb['budget'] = data_imdb['budget'].str.replace(' (estimated)', '')

data_imdb['budget'] = data_imdb['budget'].str.replace(',', '')
data_imdb['budget'] = data_imdb['budget'].str.replace(' ', '')

# Supprimer les lignes avec 'No budger' dans la colonne 'budget'
data_imdb = data_imdb[data_imdb['budget'] != 'No budget']

# Réinitialiser l'index
data_imdb.reset_index(drop=True, inplace=True)

# Afficher les premières lignes pour vérification
# display(data_imdb)


In [341]:
import re

taux_de_conversion = {
    '$': 1.0,
    '£': 1.38,  # Livre sterling (taux de change actuel)
    '€': 1.10,  # Euro (taux de change actuel)
    '₹': 0.014,  # Roupie indienne (taux de change actuel)
    'DEM': 0.62,  # Deutsche Mark (taux de change historique)
    '¥': 0.0093,  # Yen japonais (taux de change actuel)
    'FRF': 0.18,  # Franc français (taux de change historique)
    '₩': 0.00089,  # Won sud-coréen (taux de change actuel)
    'ATS': 0.090,  # Schilling autrichien (taux de change historique)
    'RUR': 0.013,  # Rouble russe (taux de change actuel)
    'DKK': 0.16,  # Couronne danoise (taux de change actuel)
    'CN¥': 0.15,  # Yuan chinois (taux de change actuel)
    'SEK': 0.12,  # Couronne suédoise (taux de change actuel)
    'ITL': 0.00061  # Lire italienne (taux de change historique)
}

# Boucler à travers les valeurs et extraire les symboles et les montants
for i, valeur in enumerate(data_imdb['budget']):
    match = re.match(r'([^$\d]+)(\d+)', valeur)
    if match:
        symbole = match.group(1).strip()
        montant = int(match.group(2))
        if symbole in taux_de_conversion:
            montant *= taux_de_conversion[symbole]
        data_imdb.at[i, 'budget'] = montant

data_imdb['budget'] = data_imdb['budget'].str.replace('$', '')


# Afficher les premières lignes pour vérification
# display(data_imdb)



In [342]:
display(data_imdb.info())
display(data_allocine.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2245 entries, 0 to 2244
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   annee              2245 non-null   int64  
 1   budget             2166 non-null   object 
 2   casting_complet    2245 non-null   object 
 3   casting_principal  2245 non-null   object 
 4   date_sortie_x      0 non-null      float64
 5   director           2245 non-null   object 
 6   duree              2245 non-null   int64  
 7   film_id            2245 non-null   object 
 8   genre              0 non-null      float64
 9   genres             2245 non-null   object 
 10  langue             2244 non-null   object 
 11  nbre_vote          2245 non-null   object 
 12  pays               2245 non-null   object 
 13  pegi               2245 non-null   object 
 14  popularite_score   2245 non-null   object 
 15  scenaristes        2245 non-null   object 
 16  score              2245 

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1726 entries, 0 to 1725
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   actors          1680 non-null   object 
 1   director        1721 non-null   object 
 2   entrees_fr      1726 non-null   int64  
 3   entrees_usa     0 non-null      float64
 4   image_urls      1726 non-null   object 
 5   nationalite     1724 non-null   object 
 6   semaine_fr      1726 non-null   object 
 7   semaine_usa     0 non-null      float64
 8   studio          1647 non-null   object 
 9   timing          1726 non-null   int64  
 10  title           1726 non-null   object 
 11  titre_original  1699 non-null   object 
dtypes: float64(2), int64(2), object(8)
memory usage: 161.9+ KB


None

In [343]:
display(data_imdb)

,annee,budget,casting_complet,casting_principal,date_sortie_x,director,duree,film_id,genre,genres,langue,nbre_vote,pays,pegi,popularite_score,scenaristes,score,titre,titre_france,date_sortie_y
0,2011,25000000,"Viola Davis,Emma Stone,Octavia Spencer,Bryce D...","Tate Taylor,Tate Taylor,Kathryn Stockett",NaN,Tate Taylor,146,tt1454029,NaN,Drama,English,492K,"United States,India",PG-13,614,Tate Taylor,8.1,The Help,NaN,02/09/2011
1,1967,3000000,"Dustin Hoffman,Anne Bancroft,Katharine Ross,Wi...","Mike Nichols,Calder Willingham,Buck Henry",NaN,Mike Nichols,106,tt0061722,NaN,"Comedy,Drama,Romance",English,288K,United States,PG,"1,516",Mike Nichols,8.0,The Graduate,NaN,04/09/1968
2,2016,35000000,"Sacha Baron Cohen,Mark Strong,Rebel Wilson,Fre...","Louis Leterrier,Writers,Sacha Baron Cohen",NaN,Louis Leterrier,83,tt3381008,NaN,"Action,Adventure,Comedy","English,Ukrainian",111K,"United Kingdom,United States",R,275,Louis Leterrier,6.2,The Brothers Grimsby,NaN,13/04/2016
3,2023,125000000,"Zachary Levi,Asher Angel,Jack Dylan Grazer,Rac...","David F. Sandberg,Writers,Henry Gayden",NaN,David F. Sandberg,130,tt10151854,NaN,"Action,Adventure,Comedy",English,124K,"United States,Germany,New Zealand,United Kingd...",PG-13,"1,137",David F. Sandberg,5.9,Shazam! Fury of the Gods,NaN,29/03/2023
4,2015,175000000,"Amy Poehler,Bill Hader,Lewis Black,Mindy Kalin...","Pete Docter,Ronnie Del Carmen,Writers",NaN,Pete Docter,95,tt2096673,NaN,"Animation,Adventure,Comedy","English,Portuguese,Latvian,Russian,Ukrainian",783K,United States,PG,638,Pete Docter,8.1,Inside Out,NaN,18/05/2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2240,2013,6000000,"Emma Stone,Stephen Merchant,Richard Gere,Liev ...","Directors,Elizabeth Banks,Steven Brill",NaN,Directors,94,tt1333125,NaN,Comedy,English,112K,United States,R,"1,362",Directors,4.3,Movie 43,NaN,19/06/2013
2241,1995,60000000,"Al Pacino,Robert De Niro,Val Kilmer,Jon Voight...","Michael Mann,Michael Mann,Stars",NaN,Michael Mann,170,tt0113277,NaN,"Action,Crime,Drama","English,Spanish",718K,United States,R,263,Michael Mann,8.3,Heat,NaN,21/02/1996
2242,2008,28000000,"Paul Rudd,Seann William Scott,Elizabeth Banks,...","David Wain,Writers,Timothy Dowling",NaN,David Wain,99,tt0430922,NaN,Comedy,English,241K,"Germany,United States",R,"2,234",David Wain,6.8,Role Models,NaN,04/02/2009
2243,2023,200000000,"Anthony Ramos,Dominique Fishback,Luna Lauren V...","Steven Caple Jr.,Writers,Joby Harold",NaN,Steven Caple Jr.,127,tt5090568,NaN,"Action,Adventure,Sci-Fi","English,Spanish",103K,United States,PG-13,624,Steven Caple Jr.,6.0,Transformers: Rise of the Beasts,NaN,07/06/2023
